In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from dataloader import load_raw, create_datasets, create_dataloaders
from measures import quantile_loss, compute_intermittent_indicators, rho_risk

import lightgbm as lgb

In [2]:
dataset_name = "M5"
method = "TreeQRs"

data_raw, data_info = load_raw(dataset_name, datasets_folder_path=os.path.join("..","data"))
adi, cv2 = compute_intermittent_indicators(data_raw, data_info['h'])
datasets = create_datasets(data_raw, data_info)

max_lag = {
    'carparts':44,
    'OnlineRetail':56,
    'Auto':16,
    'RAF':45,
    'M5':50
}

lags = np.arange(1,max_lag[dataset_name]) if dataset_name != "M5" else [1,2,3,4,5,6,7,8,9,10,15,20,25,30,50,80,100,150]
lags = np.unique(np.sort(np.concatenate([lags, np.array([data_info['h']*data_info['w']])])))
quantiles = np.array([0.5, 0.8, 0.9, 0.95, 0.99])

In [3]:
scale_factor = []
for y in datasets['valid']:
    y = np.array(y['target'])
    demand_mask = np.where(y != 0)
    scale_factor.append(np.mean(y[demand_mask]))

In [ ]:
for lag in [112]:
    print(lag)

    model_folder_name = method+"l" + str(lag) + "__" + dataset_name
    model_folder_path = os.path.join('tree-regression', model_folder_name)

    # each series is lag-embedded into a matrix at the given AR order and these matrices are stacked together to create on big matrix
    X = []
    Y = []
    Xtest = []
    Ytest = []
    for ts, sf in zip(datasets["valid"], scale_factor):
        ts = np.array(ts['target']) / sf
        for t in range(lag, len(datasets['valid'][0]['target'])):
            X.append(ts[t-lag:t])
            Y.append(ts[t])
        Xtest.append(ts[-lag:])
    for ts in datasets["test"]:
        ts = np.array(ts['target'])
        Ytest.append(ts[-data_info['h']:])
            
    X, Y = np.array(X), np.array(Y)
    Xtest, actuals = np.array(Xtest), np.array(Ytest)

    # a maximum of 1 mio samples for training
    np.random.seed(42)
    filter_max = np.random.choice(X.shape[0], size=min(X.shape[0], 1_000_000), replace=False)
    Xsub = X[filter_max]
    Ysub = Y[filter_max]

    X_df = pd.DataFrame(Xsub, columns=[f'X{i}' for i in range(1, lag + 1)])
    Y_series = pd.Series(Ysub, name='Y')
    
    split = int(X_df.shape[0]*0.85)
    train_dataset = lgb.Dataset(X_df.iloc[:split], label=Y_series.iloc[:split])
    valid_dataset = lgb.Dataset(X_df.iloc[split:], label=Y_series.iloc[split:], reference=train_dataset)
    
    models = {}
    for q in quantiles:
        models[q] = lgb.train(
            params={
                'objective': 'quantile',
                'alpha': q,
                'boosting': 'gbdt',
                'learning_rate':0.1,
                'num_leaves':31,
                'force_row_wise':True,
                'seed':42
            },
            num_boost_round=1000,
            train_set=train_dataset,
            valid_sets=[valid_dataset],
            callbacks=[
                lgb.log_evaluation(period=0, show_stdv=True),
                lgb.early_stopping(stopping_rounds=20, first_metric_only=True, verbose=True)
            ]
        )
        
    Xtest_ = Xtest
    quantile_forecasts = np.empty(shape=(Xtest.shape[0], quantiles.size, data_info['h']))
    for h in range(data_info['h']):
        for qi, q in enumerate(quantiles):
            quantile_forecasts[:,qi,h] = np.round(models[q].predict(Xtest_) * scale_factor)
        Xtest_ = np.concatenate([Xtest_[:,1:lag], models[0.5].predict(Xtest_).reshape(-1,1)], axis=1)
    
    if not os.path.exists(path=model_folder_path):
        os.makedirs(model_folder_path)
    if lag==1: np.save(os.path.join(model_folder_path,"actuals.npy"), actuals)
    np.save(os.path.join(model_folder_path,"qforecasts.npy"), quantile_forecasts)

In [5]:
subset="intermittent_and_lumpy"
if subset == "intermittent":
    filter, filter_label = np.logical_and(adi >= 1.32, cv2 < .49), "intermittent"
elif subset == "intermittent_and_lumpy":
    filter, filter_label = adi >= 1.32, "intermittent_and_lumpy"
elif subset == "all":
    filter, filter_label = np.tile(True, adi.size), "all"

In [6]:
tmp = np.empty(shape=(len(datasets['test']), len(datasets['valid'][0]['target']), quantiles.size))
for i in range(len(datasets['test'])):
    tmp[i, :] = np.round(np.quantile(datasets['valid'][i]['target'], q=quantiles))
res_base_scale_tmp = []
for i in range(len(datasets['test'])):
    res_base_scale_tmp.append(quantile_loss(
        np.array(datasets['valid'][i]['target']).reshape(1,-1), 
        tmp[i].reshape(1,tmp[i].shape[0],tmp[i].shape[1]), 
        quantiles, avg=False))
res_base_scale = {}
for q in ['QL50','QL80','QL90','QL95','QL99']:
    res_base_scale[q] = np.mean(np.vstack([res_base_scale_tmp[i][q] for i in range(len(datasets['test']))]), axis=1)

scale = True
fscale = lambda x, q: x / res_base_scale[q][filter, np.newaxis] if scale else x

path = os.path.join("tree-regression")
names = [folder for folder in os.listdir(path) 
                  if os.path.isdir(os.path.join(path, folder))]
names_sub = [x for x in names if (x.split('__')[1] == dataset_name) and (x.split('__')[0].split('l')[0] == method)]
M = {}
actuals = np.load(os.path.join(path, method+'l1__'+dataset_name, "actuals.npy"))[filter]
for n in names_sub:
    quantile_forecasts = np.load(os.path.join(path, n, "qforecasts.npy"))[filter]
    M[int(n.split('l')[1].split('__')[0])] = quantile_loss(actuals, np.transpose(quantile_forecasts, (0,2,1)), quantiles, avg=False)
M = {key: M[key] for key in sorted(M.keys())}

for k in M.keys():
    for l in M[k].keys():
        M[k][l] = fscale(M[k][l], l)

# Local methods
baseline_path = os.path.join(os.path.expanduser("~/switchdrive"), "iTS", "trained_models_baselines")
baselines_name = [folder for folder in os.listdir(baseline_path) 
                  if os.path.isdir(os.path.join(baseline_path, folder)) and os.path.exists(os.path.join(baseline_path, folder, 'metrics.json'))]
baselines_name_sub = [x for x in baselines_name if x.split('__')[1] == dataset_name]

iETS_actuals = np.load(os.path.join(baseline_path, np.array(baselines_name_sub)[["iETS" in x for x in baselines_name_sub]][0], "actuals.npy"))[filter]
iETS_quantile_forecasts = np.load(os.path.join(baseline_path, np.array(baselines_name_sub)[["iETS" in x for x in baselines_name_sub]][0], "qforecasts.npy"))[filter]
iETS = quantile_loss(iETS_actuals, iETS_quantile_forecasts, quantiles, avg=False)
EmpQ_actuals = np.load(os.path.join(baseline_path, np.array(baselines_name_sub)[["EmpQ" in x for x in baselines_name_sub]][0], "actuals.npy"))[filter]
EmpQ_quantile_forecasts = np.load(os.path.join(baseline_path, np.array(baselines_name_sub)[["EmpQ" in x for x in baselines_name_sub]][0], "qforecasts.npy"))[filter]
EmpQ = quantile_loss(EmpQ_actuals, EmpQ_quantile_forecasts, quantiles, avg=False)

assert iETS.keys() == EmpQ.keys()
for k in iETS.keys():
    iETS[k] = fscale(iETS[k],k)
    EmpQ[k] = fscale(EmpQ[k],k)

In [7]:
import pickle as pkl
pkl.dump(M, open(os.path.join('cache_global', method+'_'+dataset_name+".pkl"), 'wb'))
pkl.dump(iETS, open(os.path.join('cache_global', "iETS"+'_'+dataset_name+".pkl"), 'wb'))
pkl.dump(EmpQ, open(os.path.join('cache_global', "EmpQ"+'_'+dataset_name+".pkl"), 'wb'))

In [ ]:
Q = list(M[lags[0]].keys())
fig, axs = plt.subplots(1,len(Q), figsize=(10,5), sharey=True)
colors = ['#87CEFA','#00BFFF','#1E90FF','#4682B4','#191970']
for ax, c, q_ in zip(axs, colors, Q):
    tmp = [np.mean(M[l][q_]) for l in lags]
    ax.hlines(y=np.mean(EmpQ[q_]), xmin=1, xmax=lags[-1], color=c, linestyle=':', label="EmpQ")
    ax.hlines(y=np.mean(iETS[q_]), xmin=1, xmax=lags[-1], color=c, linestyle='--', label="iETS")
    ax.plot(lags, tmp, color=c, label="TreeQR")
    ax.set_title(q_)
axs[2].set_xlabel('LAG')
axs[0].set_ylabel('QL')
axs[-1].legend(loc="upper right")
plt.suptitle(dataset_name)
plt.tight_layout()
plt.show()